In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv
/kaggle/input/gujarat-biomass-production/Biomass_History.csv


In [2]:
distance=pd.read_csv("/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv",index_col=0)
biomass=pd.read_csv('/kaggle/input/gujarat-biomass-production/Biomass_History.csv')


In [51]:
distance.shape

(2418, 2418)

In [3]:
val=biomass['2010'].values

In [4]:
wt=val[:,np.newaxis]+val[np.newaxis,:]

In [5]:
dist=distance.values.copy()

In [54]:
distance_transform[18][1432]*0.001*167.43

76.148285781

In [6]:
for i in range(distance.shape[0]):
    wt[i][i]=0
    dist[i][i]=1

In [35]:
((distance==0).sum()!=1)[((distance==0).sum()!=1)]

205     True
206     True
276     True
277     True
279     True
280     True
664     True
665     True
725     True
726     True
1349    True
1414    True
1415    True
1416    True
dtype: bool

In [39]:
distance.values[205][206]

0.0

In [7]:
weight=wt/dist

/tmp/ipykernel_32/799300043.py:1: RuntimeWarning: divide by zero encountered in divide
  weight=wt/dist


In [8]:
weight[weight==np.inf]=1e9

In [66]:
class DSU:
    def __init__(self):
        self.p = {}
        self.count = 0
        self.biomass={}
        self.validnode=list(range(2418))
        self.clusture=[]
    
    def add(self, x,val):
        if x in self.p: return
        self.p[x] = x
        self.count += 1
        self.biomass[x]=val
    
    def find(self, x):
        if x != self.p[x]:
            self.p[x] = self.find(self.p[x])
        return self.p[x]
    
    def union(self, x, y,dist,value):
        total=self.biomass[x]+self.biomass[y]
        for node in self.validnode :
            if node!=x and node!=y:
                dist[x][node]=min(dist[x][node],dist[y][node])
                dist[node][x]=min(dist[node][x],dist[node][y])
                if(dist[x][node]==0):
                    value[x][node]=1e9
                else:
                    value[x][node]=(self.biomass[node]+total)/dist[x][node]
                if(dist[node][x]==0):
                    value[node][x]=1e9
                else:
                    value[node][x]=(self.biomass[node]+total)/dist[node][x]
        for index in range(2418):
            value[index][y]=-1
            value[y][index]=-1
        self.validnode.remove(y)
        
#         xp, yp = self.find(x), self.find(y)
#         if xp == yp: return False
#         self.p[xp] = yp
#         self.count -= 1
#         return True
    
    def is_connected(self, x, y):
        if x in self.p and y in self.p:
            return self.find(x) == self.find(y)
        else:
            return False

In [76]:
dsu=DSU()
distance_transform_copy=distance_transform.copy()
weight_copy=weight.copy()
for i in range(2418):
    dsu.add(i,val[i])
flag=1    
while (flag):
    ind = np.unravel_index(np.argmax(weight_copy, axis=None), weight_copy.shape)
    x=ind[0]
    y=ind[1]
    if(weight_copy[ind]==-1):
        break
    if(dsu.biomass[x]+dsu.biomass[y]<20000):
        dsu.union(x,y,distance_transform_copy,weight_copy)
    else :
        weight_copy[x][y]=-1
        weight_copy[y][x]=-1

In [70]:
len(dsu.validnode)

2418

In [9]:
class DSU:
    def __init__(self):
        self.p = {}
        self.count = 0
        self.biomass={}
        self.partialjoin=[]
    
    def add(self, x,val):
        if x in self.p: return
        self.p[x] = x
        self.count += 1
        self.biomass[x]=val
    
    def find(self, x):
        if x != self.p[x]:
            self.p[x] = self.find(self.p[x])
        return self.p[x]
    
    def union(self, x, y):
        xp, yp = self.find(x), self.find(y)
        if xp == yp: return False
        self.p[xp] = yp
        self.biomass[yp]+=self.biomass[xp]
        self.count -= 1
        return True
    
    def partialunion(self,x,y):
        xp, yp = self.find(x), self.find(y)
        val=20000-self.biomass[yp]
        self.biomass[yp]+=val
        self.biomass[xp]-=val
        self.partialjoin.append([val,xp,yp])
        return val
    
    def is_connected(self, x, y):
        if x in self.p and y in self.p:
            return self.find(x) == self.find(y)
        else:
            return False

In [11]:
edges=[]
for i in range(2418):
    for j in range(2418):
        if i!=j:
            edges.append([weight[i][j],i,j])
edges=sorted(edges,key=lambda l:l[0], reverse=True)

In [24]:
len(edges)

5844306

In [13]:
dsu=DSU()
for i in range(2418):
    dsu.add(i,val[i])
for index in range(len(edges)):
    edge=edges[index]
    value=edge[0]
    i=edge[1]
    j=edge[2]
    if dsu.biomass[i]<dsu.biomass[j] and dsu.p[i]==i and dsu.p[j]==j and dsu.biomass[j]<20000:
        if dsu.biomass[j]+dsu.biomass[i]<=20000:
            dsu.union(i,j)
        else:
            reduce=dsu.partialunion(i,j)
            for index2 in range(index+1,len(edges)):
                edge2=edges[index2]
                if(edge2[1]==i or edge2[2]==i) and dist[edge2[1]][edge2[2]]!=0 :
                    edges[index][0]-=(reduce/dist[edge2[1]][edge2[2]])
            edges=edges[:index+1]+sorted(edges[index+1:],key=lambda l:l[0], reverse=True)

In [15]:
dsu.partialjoin

[[3246.081634199996, 1974, 1805],
 [2871.861648987997, 1147, 858],
 [319.1708449100006, 1338, 1341],
 [3294.8215942200004, 2040, 1866],
 [241.67145543999868, 1147, 1584],
 [7729.66272894, 551, 606],
 [2165.7439685479985, 929, 1147],
 [247.1500449319974, 884, 1122],
 [6692.938037900001, 1367, 1608],
 [3936.6592870839995, 1453, 1742],
 [4109.767619526998, 1653, 942],
 [8289.891702893996, 1653, 929],
 [817.7424906489869, 1967, 2214],
 [1582.7093725000013, 337, 348],
 [715.6543787009978, 596, 337],
 [350.13371748997815, 2046, 1504]]

In [27]:
cluster=[]
for i in range(2418):
    cluster.append(dsu.find(i))

In [28]:
np.unique(cluster)

array([ 337,  348,  606,  858,  929,  942, 1122, 1147, 1341, 1504, 1584,
       1608, 1742, 1805, 1866, 2046, 2144, 2214])

In [31]:
for i in np.unique(cluster):
    print(i,dsu.biomass[i])

337 20000.0
348 20000.0
606 20000.0
858 20000.0
929 20000.0
942 20000.0
1122 20000.0
1147 20000.0
1341 20000.0
1504 20000.0
1584 20000.0
1608 20000.0
1742 20000.0
1805 20000.0
1866 20000.0
2046 16865.862260834023
2144 12296.781086487996
2214 20000.0


In [29]:
import plotly.express as px

fig = px.scatter_mapbox(biomass, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index", "2010"],
                        color=cluster,
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [32]:
import plotly.express as px

color=cluster.copy()
for index in range(len(color)):
    if color[index]==2046 or color[index]==2144:
        color[index]=-1
fig = px.scatter_mapbox(biomass, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index", "2010"],
                        color=color,
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()